In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.fft import fft
from IPython.display import Audio, display
from zvon import *

In [2]:
def show(y):
    x = range(len(y))
    print('min:', np.min(y), ', max:', np.max(y))
    plt.plot(x, y)
    plt.show()

def log10(x):
    return np.log10(x.clip(min=1e-30))

def show_fft(y, max_freq=15000):
    fft_y = 20 * log10(np.abs(fft(y)) / SR)
    plt.plot(range(len(y)), fft_y)
    plt.xlim(0, max_freq)
    plt.ylim(-120, 0)
    plt.show()

def save_audio(y, filename='test.wav'):
    audio = Audio(y, rate=SR)
    display(audio)
    open(filename, "wb").write(audio.data)

def save_audio_stereo(y, filename='test.wav'):
    left, right = np.array([v[0] for v in y]), np.array([v[1] for v in y])
    save_audio([left, right], filename)

In [3]:
NOTES = {
    'c-': 0,
    'c#': 1,
    'd-': 2,
    'd#': 3,
    'e-': 4,
    'f-': 5,
    'f#': 6,
    'g-': 7,
    'g#': 8,
    'a-': 9,
    'a#': 10,
    'b-': 11
}

def note_freq(note):
    n, o = note[:2].lower(), int(note[2])
    return midi_freq(NOTES[n] + 12 * (o + 2))

def change(stack, params):
    for box_idx, param, val1, val2 in params:
        box = stack[box_idx]
        box.change(box.state, param, val1, val2)

def play_song(channels, song, tick):
    samples = []
    for row in song:
        for chan, params in zip(channels, row):
            if chan.is_on:
                change(chan.stack, params)
        samples += mix_process(channels, len(row), tick, 1)
    return samples

In [4]:
class test_square_state:
    __slots__ = ('ph1', 'ph2', 'freq', 'vol', 'env')

def test_square_init(s):
    s.ph1 = Phasor()
    phasor_init(s.ph1)
    s.ph2 = Phasor() 
    phasor_init(s.ph2)
    s.freq = 0
    s.vol = 0
    s.env = Env()
    env_init(s.env, [sec(0.01), sec(0.5)], 0, [1, 0])

def test_square_change(s, param, val1, val2):
    if param == 'ZV_NOTE_ON':
        s.freq = val1
        env_reset(s.env)
    elif param == 'ZV_VOLUME':
        s.vol = val1

def test_square_next(s, l):
    a = square(phasor_next(s.ph1, s.freq) + 2 * math.sin(phasor_next(s.ph2, 4)), 0.5)
    return s.vol * a * env_next(s.env)

def test_square_box(s):
    print(dir(s))
    s.change = test_square_change
    s.mono = test_square_next
    s.state = test_square_state()
    s.stereo = None
    test_square_init(s.state)

class test_saw_state:
    __slots__ = ('ph1', 'ph2', 'ph3', 'freq', 'vol', 'env')

def test_saw_init(s):
    s.ph1 = Phasor()
    phasor_init(s.ph1)
    s.ph2 = Phasor()        
    phasor_init(s.ph2)
    s.ph3 = Phasor()        
    phasor_init(s.ph3)
    s.freq = 0
    s.vol = 0
    s.env = Env()
    env_init(s.env, [sec(0.01), sec(0.1)], 0, [1, 0.5])

def test_saw_change(s, param, val1, val2):
    if param == 'ZV_NOTE_ON':
        s.freq = val1
        env_reset(s.env)
    elif param == 'ZV_VOLUME':
        s.vol = val1

def test_saw_next(s, l):
    mod = 0.2 + abs(1 + math.sin(phasor_next(s.ph2, 1))) * 0.3
    a = saw(phasor_next(s.ph1, s.freq) + 2 * math.sin(phasor_next(s.ph3, 4)), mod)
    return s.vol * a * env_next(s.env)

def test_saw_box(s):
    s.change = test_saw_change
    s.mono = test_saw_next
    s.state = test_saw_state()
    s.stereo = None
    test_saw_init(s.state)

class test_delay_state:
    __slots__ = ('dly',)

def test_delay_init(s):
    s.dly = Delay()
    delay_init(s.dly, sec(0.5), 0.5, 0.5)

def test_delay_change(s, param, val1, val2):
    if param == 'ZV_VOLUME':
        s.dly.level = val1

def test_delay_next(s, l):
    return delay_next(s.dly, l)

def test_delay_box(s):
    s.change = test_delay_change
    s.mono = test_delay_next
    s.state = test_delay_state()
    s.stereo = None
    test_delay_init(s.state)

class box_overdrive_state:
    __slots__ = ('drive',)

def box_overdrive_init(s):
    s.drive = 1
    s.vol = 1

def box_overdrive_change(s, param, val):
    if param == 'ZV_VOLUME':
        s.vol = val
    elif param == 'ZV_DRIVE':
        s.drive = val

def box_overdrive_mono(s, l):
    return s.vol * overdrive(l, s.drive)

def box_overdrive(s):
    s.change = box_overdrive_change
    s.mono = box_overdrive_mono
    s.state = box_overdrive_state()
    s.stereo = None
    box_overdrive_init(s.state)

In [5]:
mix_init(Channels)
chan_push(Channels[0], test_square_box)
chan_push(Channels[0], test_delay_box)
chan_set(Channels[0], True, 1, 0)

def note_on(box_idx, note):
    return (box_idx, 'ZV_NOTE_ON', note_freq(note), 0)

def volume(box_idx, vol):
    return (box_idx, 'ZV_VOLUME', vol, 0)

song = [
    [(note_on(0, 'c-3'), volume(0, 1))],
    [(), ()],
    [(note_on(0, 'g-3'), volume(0, 1))],
    [(), ()],
    [(note_on(0, 'e-3'), volume(0, 1))],
    [(), ()],
    [(note_on(0, 'g-3'), volume(0, 1))],
    [(), ()],
    [(note_on(0, 'c-4'), volume(0, 1))],
    [(), ()],
    [(note_on(0, 'g-3'), volume(0, 1))],
    [(), ()],
    [(note_on(0, 'e-3'), volume(0, 1))],
    [(), ()],
    [(note_on(0, 'g-3'), volume(0, 1))],
    [(), ()]    
]

y = play_song(Channels, song, sec(1/8))

show(y)
save_audio_stereo(y)

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'change', 'mono', 'state', 'stereo']


AttributeError: 'Box' object has no attribute 'is_stereo'

In [ ]:
def play(t):
    return overdrive(math.sin(hertz(t, 440)), 10)

y = [play(t) for t in range(sec(1))]

show(y)
save_audio(y)

NameError: name 'overdrive' is not defined